# Cartpole Reinforcement Learning

### Importing the necessary Modules

In [103]:
import gym
import numpy as np

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent,DDPGAgent,CEMAgent
from rl.policy import BoltzmannQPolicy
# from rl.random import OrnsteinUhlenbeckProcess
from rl.memory import SequentialMemory,EpisodeParameterMemory

### Creating the environement 

In [104]:
env = gym.make("Ant-v4")

c:\Users\TRETEC\anaconda3\envs\ai\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\TRETEC\anaconda3\envs\ai\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


### Analyzing the cartpole environement

In [105]:
states = env.observation_space.shape[0]
actions = env.action_space.shape[0]

In [106]:
print(f"number of states : {states}")
print(f"number of actions : {actions}")


number of states : 27
number of actions : 8


In [135]:
env.observation_space.sample()

array([ 0.72440271, -1.47296788,  0.72975852, -0.52921578,  1.15051308,
        1.81105914, -0.76127394, -0.49708759, -1.61719742,  0.32503977,
        0.77108341, -1.93947159, -0.96565965, -0.56417428, -1.15575783,
       -0.05952307, -0.09246969,  0.43618037, -0.9690972 ,  0.19809041,
       -0.97633534, -1.65211282, -0.12409619, -0.1062029 ,  1.67592125,
       -0.4974922 ,  0.32372578])

In [134]:
env.action_space.sample()

array([ 0.12594579, -0.9639627 ,  0.14355478, -0.8361521 ,  0.36573464,
        0.2651483 ,  0.25457457,  0.46200183], dtype=float32)

### Testing the Environement on random samples

In [109]:
# episodes = 10
# for episode in range(1,episodes+1):
#     state = env.reset()
#     done = False
#     score=0

#     while not done:
#         action = env.action_space.sample()
#         _,reward,done,_ = env.step(action)
#         score +=reward
#         env.render()

#     print(f"Episode {episode}, score {score}")

# env.close()




### Generating the training the neural network on the environement

In [151]:
def build_model(states,actions):
    model = Sequential([
        Flatten(input_shape=(1,27)),
        Dense(64,activation="relu"),
        Dense(64,activation="relu"),
        Dense(8,activation="tanh"),
        # Flatten(),
    ])
    return model
# model = Sequential([
#     observation_input = Input(shape=(1,) + env.observation_space.shape, name='observation_input')

# # Define your custom neural network model
# x = Flatten()(observation_input)
#     Flatten(input_shape=(1,states)),
#     Dense(64, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(actions, activation='tanh')
# ])

In [152]:
# Define the memory
memory = SequentialMemory(limit=100000, window_length=1)

# Define the policy
policy = BoltzmannQPolicy()

In [153]:
model = build_model(states,actions)
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_27 (Flatten)         (None, 27)                0         
_________________________________________________________________
dense_78 (Dense)             (None, 64)                1792      
_________________________________________________________________
dense_79 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_80 (Dense)             (None, 8)                 520       
Total params: 6,472
Trainable params: 6,472
Non-trainable params: 0
_________________________________________________________________


In [156]:
t = np.array([1,2,3,4,5,6,7,8])
t.shape

(8,)

In [154]:
# agent = CEMAgent(model=build_model(), memory=memory, nb_actions=env.action_space.shape[0])
agent = DQNAgent(model=model, memory=memory, policy=policy,
               nb_actions=actions, nb_steps_warmup=10,
               target_model_update=1e-2)

In [155]:
agent.compile(Adam(lr=1e-3), metrics=['mae'])
agent.fit(env,nb_steps=100000,visualize=False,verbose=1)

C:\Users\TRETEC\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Training for 100000 steps ...
Interval 1 (0 steps performed)


C:\Users\TRETEC\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


ValueError: Action dimension mismatch

In [33]:
results =agent.test(env,nb_episodes=10,visualize=True)
print(np.mean(results.history["episode_reward"]))
env.close()


Testing for 10 episodes ...


c:\Users\TRETEC\anaconda3\envs\ai\lib\site-packages\gym\core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
500.0
